In [ ]:
# 필요한 module import 및 trainset, testset 만들기

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init

import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import torchvision.models as models

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

trainset = dset.STL10(root="../", split = "train", transform = transforms.ToTensor(), download = True)
testset = dset.STL10(root="../", split = "test", transform = transforms.ToTensor(), download = True)

# 이미지 96x96, color

  0%|          | 0/2640397119 [00:00<?, ?it/s]

Extracting ../stl10_binary.tar.gz to ../
Files already downloaded and verified


Problem 2 - 시도 1. Alexnet 구현


In [ ]:
# 수업시간에 구현했던 모델이라 선택함. 주어진 데이터의 형태(3 x 96 x 96)에 맞게 변형하여 구현함.

batch_size_1 = 32
learning_rate_1 = 0.0001
num_epoch = 50

trainloader_1 = DataLoader(trainset, batch_size = batch_size_1)
testloader_1 = DataLoader(testset, batch_size = batch_size_1)

In [ ]:
class AlexNet_model1(nn.Module):
    def __init__(self):
        super(AlexNet_model1, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 7), #[batch_size, 64, 90, 90]
            nn.ReLU(),
            nn.MaxPool2d(2,2), #[batch_size, 64, 945, 45]
            nn.Conv2d(64, 192, 7),#[batch_size, 192, 39, 39]
            nn.ReLU(),
            nn.MaxPool2d(2,2), #[batch_size, 192, 19, 19]
            nn.Conv2d(192, 384, 7),#[batch_size, 384, 13, 13]
            nn.ReLU(),
            nn.Conv2d(384, 256, 5),#[batch_size, 256, 9, 9]
            nn.ReLU(),
            nn.Conv2d(256, 256, 3),#[batch_size, 256, 7, 7]
            nn.ReLU(),
            nn.MaxPool2d(2,2)) #[batch_size, 256, 3, 3]
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 3 * 3,1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024,512),
            nn.ReLU(),
            nn.Linear(512, 10))
    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, 256 * 3 * 3)
        x = self.classifier(x)
        return x

In [ ]:
model1 = AlexNet_model1().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model1.parameters(), lr = learning_rate_1)

In [ ]:
for i in range(num_epoch):
    for j, [image, label] in enumerate(trainloader_1):
        x = image.to(device)
        y_ = label.to(device)

        optimizer.zero_grad()
        output = model1.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
    if i % 10 == 0:
        print(loss)

tensor(1.9206, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.3028, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5702, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0045, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0760, device='cuda:0', grad_fn=<NllLossBackward0>)


In [ ]:
correct = 0
total = 0

with torch.no_grad():
    model1.eval()
    for data in testloader_1:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model1(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print("model1 accuracy : ", 100 * correct / total)

model1 accuracy :  55.0125


Problem 2 - 시도 2. Alexnet 구현


*   padding 사용


In [ ]:
# 첫 번째 시도에서 padding을 이용하지 않아 padding을 사용했을 때의 성능의 변화가 궁금함.

batch_size_2 = 32
learning_rate_2 = 0.0001
num_epoch = 50

trainloader_2 = DataLoader(trainset, batch_size = batch_size_2)
testloader_2 = DataLoader(testset, batch_size = batch_size_2)

In [ ]:
class AlexNet_model2(nn.Module):
    def __init__(self):
        super(AlexNet_model2, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 7), #[batch_size, 64, 90, 90]
            nn.ReLU(),
            nn.MaxPool2d(2,2), #[batch_size, 64, 45, 45]
            nn.Conv2d(64, 192, 7, padding = 1), #[batch_size, 192, 41, 41]
            nn.ReLU(),
            nn.MaxPool2d(2,2), #[batch_size, 192, 20, 20]
            nn.Conv2d(192, 384, 7, padding = 1), #[batch_size, 384, 16, 16]
            nn.ReLU(),
            nn.Conv2d(384, 256, 5, padding = 1), #[batch_size, 256, 14, 14]
            nn.ReLU(),
            nn.Conv2d(256, 256, 3), #[batch_size, 256, 12, 12]
            nn.ReLU(),
            nn.MaxPool2d(2,2)) #[batch_size, 256, 6, 6]
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 6 * 6,1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024,512),
            nn.ReLU(),
            nn.Linear(512, 10))
    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, 256 * 6 * 6)
        x = self.classifier(x)
        return x

In [ ]:
model2 = AlexNet_model2().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr = learning_rate_2)

In [ ]:
for i in range(num_epoch):
    for j, [image, label] in enumerate(trainloader_2):
        x = image.to(device)
        y_ = label.to(device)

        optimizer.zero_grad()
        output = model2.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
    if i % 10 == 0:
        print(loss)

tensor(1.8699, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.9101, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0496, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0125, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0043, device='cuda:0', grad_fn=<NllLossBackward0>)


In [ ]:
correct = 0
total = 0

with torch.no_grad():
    model2.eval()
    for data in testloader_2:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model2(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print("model2 accuracy : ", 100 * correct / total)

model2 accuracy :  57.2


Problem 2 - 시도3. Alexnet 구현


*   padding 사용
*   배치사이즈 키움


In [ ]:
# problem 1에서 배치사이즈를 키웠을때 성능이 향상되어 problem2에서도 배치사이즈가 커졌을 때 성능이 향상될지 궁금함. 

batch_size_3 = 256
learning_rate_3 = 0.0001
num_epoch = 50

trainloader_3 = DataLoader(trainset, batch_size = batch_size_3)
testloader_3 = DataLoader(testset, batch_size = batch_size_3)

In [ ]:
class AlexNet_model3(nn.Module): #Alexnet_model2와 구현 방식은 동일함
    def __init__(self):
        super(AlexNet_model3, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 7),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(64, 192, 7, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(192, 384, 7, padding = 1),
            nn.ReLU(),
            nn.Conv2d(384, 256, 5, padding = 1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3),
            nn.ReLU(),
            nn.MaxPool2d(2,2))
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 6 * 6,1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024,512),
            nn.ReLU(),
            nn.Linear(512, 10))
    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, 256 * 6 * 6)
        x = self.classifier(x)
        return x

In [ ]:
model3 = AlexNet_model3().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model3.parameters(), lr = learning_rate_3)

In [ ]:
for i in range(num_epoch):
    for j, [image, label] in enumerate(trainloader_3):
        x = image.to(device)
        y_ = label.to(device)

        optimizer.zero_grad()
        output = model3.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
    if i % 10 == 0:
        print(loss)

tensor(2.2415, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.5325, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.2187, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.8718, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5827, device='cuda:0', grad_fn=<NllLossBackward0>)


In [ ]:
correct = 0
total = 0

with torch.no_grad():
    model3.eval()
    for data in testloader_3:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model3(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print("model3 accuracy : ", 100* correct / total)

model3 accuracy :  55.9375


Problem 2 - 시도 4. Alexnet 구현 


*   padding 사용
*   Conv2d, Maxpool 추가



In [ ]:
# 처음 Alexnet에서 합성곱, 최대풀링, ReLU층을 추가하면 성능이 더 향상될까 궁금하여 추가해봄.

batch_size_4 = 32
learning_rate_4 = 0.0001
num_epoch = 50

trainloader_4 = DataLoader(trainset, batch_size = batch_size_4)
testloader_4 = DataLoader(testset, batch_size = batch_size_4)

In [ ]:
class AlexNet_model4(nn.Module):
    def __init__(self):
        super(AlexNet_model4, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 7), #[batch_size, 64, 90, 90]
            nn.ReLU(),
            nn.MaxPool2d(2,2), #[batch_size, 64, 45, 45]
            nn.Conv2d(64, 192, 7, padding = 1), #[batch_size, 192, 41, 41]
            nn.ReLU(),
            nn.MaxPool2d(2,2), #[batch_size, 192, 20, 20]
            nn.Conv2d(192, 384, 7, padding = 1), #[batch_size, 384, 16, 16]
            nn.ReLU(),
            nn.Conv2d(384, 768, 3, padding = 1), #추가 [batch_size, 768, 16, 16]
            nn.ReLU(),
            nn.MaxPool2d(2,2), #추가 [batch_size, 768, 8, 8]
            nn.Conv2d(768, 384, 3, padding = 1), #추가 [batch_size, 384, 8, 8]
            nn.ReLU(),
            nn.Conv2d(384, 256, 5, padding = 1),  #[batch_size, 256, 6, 6]
            nn.ReLU(),
            nn.Conv2d(256, 256, 3), #[batch_size, 256, 6, 6]
            nn.ReLU(), #추가
            nn.MaxPool2d(2,2), #추가 [batch_size, 256, 3, 3]
            nn.ReLU(),
            nn.MaxPool2d(2,2)) #[batch_size, 256, 1, 1]
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 10))
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [ ]:
model4 = AlexNet_model4().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model4.parameters(), lr = learning_rate_4)

In [ ]:
for i in range(num_epoch):
    for j, [image, label] in enumerate(trainloader_4):
        x = image.to(device)
        y_ = label.to(device)

        optimizer.zero_grad()
        output = model4.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
    if i % 10 == 0:
        print(loss)

tensor(2.3764, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.1942, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6200, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1498, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0031, device='cuda:0', grad_fn=<NllLossBackward0>)


In [ ]:
correct = 0
total = 0

with torch.no_grad():
    model4.eval()
    for data in testloader_4:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model4(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print("model4 accuracy : ", 100 * correct / total)

model4 accuracy :  58.1125


# Problem 2 결과
최종 성능 58.1125%로 model4 (Alexnet 변형, padding = 1, batch size = 32, 합성곱 및 최대풀링층 추가)에서 가장 높은 성능을 기록함.